In [1]:
import pandas as pd
import paths, data

In [2]:
data_location = "s3://prm-gp2gp-data-sandbox-dev/transfers-sample-2/"
transfers_sample_file = data_location + "12-2020-transfers.parquet"
asid_lookup_file = data_location + "asidLookup.csv"
asid_lookup = pd.read_csv(asid_lookup_file)
transfers = pd.read_parquet(transfers_sample_file)

In [3]:
response_codes = pd.read_csv(data.gp2gp_response_codes.path)
codes = response_codes[["ErrorCode", "ErrorName"]]
transfers = transfers.merge(codes, left_on='final_error_code',right_on='ErrorCode',how='left')
transfers = transfers.rename({'ErrorName': 'requestor_error_name', 'ErrorCode': 'requestor_error'}, axis=1)
transfers = transfers.merge(codes, left_on='sender_error_code',right_on='ErrorCode',how='left')
transfers = transfers.rename({'ErrorName': 'sender_error_name', 'ErrorCode': 'sender_error'}, axis=1)

In [4]:
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    None: "Unknown"
}

lookup = asid_lookup[["ASID", "MName"]]
transfers = transfers.merge(lookup, left_on='requesting_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'requesting_supplier', 'ASID': 'requesting_supplier_asid'}, axis=1)
transfers = transfers.merge(lookup, left_on='sending_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'sending_supplier', 'ASID': 'sending_supplier_asid'}, axis=1)

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

In [5]:
eight_days_in_seconds = 60* 60 * 24 * 8

# Assumption (that is true for this dataset) - the data was generated more than 8 days after the end of december 2020
# Therefore: All pending transfers have been pending for at least 8 days and will have triggered paper.

transfer_finished_beyond_8_days = transfers["sla_duration"] > eight_days_in_seconds
transfer_not_integrated = transfers["status"] != "INTEGRATED"
transfers_paper_fallback = transfer_finished_beyond_8_days | transfer_not_integrated
transfers["paper_fallback"] = transfers_paper_fallback
transfers["paper_fallback"].value_counts()

False    150558
True      26110
Name: paper_fallback, dtype: int64

In [6]:
total_transfers_per_supplier_pathway = transfers.pivot_table(
    index= "sending_supplier",
    columns="requesting_supplier",
    values="paper_fallback",
    aggfunc="count"
)
total_transfers_per_supplier_pathway

requesting_supplier,EMIS,Microtest,TPP,Unknown,Vision
sending_supplier,,,,,
EMIS,103478.0,87.0,35886.0,92.0,2050.0
Microtest,69.0,17.0,58.0,NaN,2.0
TPP,30424.0,52.0,5.0,84.0,454.0
Unknown,174.0,NaN,115.0,NaN,3.0
Vision,2547.0,1.0,531.0,NaN,539.0


In [7]:
total_fallback_per_supplier_pathway = transfers.pivot_table(
    index= "sending_supplier",
    columns="requesting_supplier",
    values="paper_fallback",
    aggfunc="sum"
)
total_fallback_per_supplier_pathway

requesting_supplier,EMIS,Microtest,TPP,Unknown,Vision
sending_supplier,,,,,
EMIS,11365.0,73.0,5034.0,9.0,1110.0
Microtest,62.0,17.0,52.0,NaN,2.0
TPP,5884.0,52.0,5.0,53.0,402.0
Unknown,48.0,NaN,27.0,NaN,0.0
Vision,1329.0,1.0,271.0,NaN,314.0


In [8]:
total_fallback_per_supplier_pathway / total_fallback_per_supplier_pathway.sum().sum() * 100

requesting_supplier,EMIS,Microtest,TPP,Unknown,Vision
sending_supplier,,,,,
EMIS,43.527384,0.279586,19.279969,0.034470,4.251245
Microtest,0.237457,0.065109,0.199157,NaN,0.007660
TPP,22.535427,0.199157,0.019150,0.202987,1.539640
Unknown,0.183838,NaN,0.103409,NaN,0.000000
Vision,5.090004,0.003830,1.037917,NaN,1.202604


In [9]:
chance_of_fallback_per_supplier_pathway = transfers.pivot_table(
    index= "sending_supplier",
    columns="requesting_supplier",
    values="paper_fallback",
    aggfunc="mean"
)

pct_chance_of_fallback_per_supplier_pathway = (chance_of_fallback_per_supplier_pathway * 100).round(decimals=2)

pct_chance_of_fallback_per_supplier_pathway

requesting_supplier,EMIS,Microtest,TPP,Unknown,Vision
sending_supplier,,,,,
EMIS,10.98,83.91,14.03,9.78,54.15
Microtest,89.86,100.00,89.66,NaN,100.00
TPP,19.34,100.00,100.00,63.10,88.55
Unknown,27.59,NaN,23.48,NaN,0.00
Vision,52.18,100.00,51.04,NaN,58.26


In [10]:
paper_fallback_transfers = transfers[transfers_paper_fallback]

In [11]:
has_sender_error = paper_fallback_transfers["sender_error"].notna()
has_requestor_error = paper_fallback_transfers["requestor_error"].notna()
is_pending_error = paper_fallback_transfers["status"] == "PENDING_WITH_ERROR"
is_integrated = paper_fallback_transfers["status"] == "INTEGRATED"
is_pending = paper_fallback_transfers["status"] == "PENDING"

In [12]:
paper_fallback_transfers[is_pending_error]["sender_error"].value_counts(dropna=False)

14.0    1343
30.0    1129
10.0     459
20.0     245
6.0      162
7.0       68
19.0      55
23.0      31
99.0      15
NaN       13
24.0      11
21.0       1
Name: sender_error, dtype: int64

In [13]:
paper_fallback_transfers["status"].value_counts()

INTEGRATED            13343
PENDING                5423
FAILED                 3812
PENDING_WITH_ERROR     3532
Name: status, dtype: int64

In [14]:
paper_fallback_transfers.loc[:, "fallback_reason"] = "other uncatagorised failure"
paper_fallback_transfers.loc[is_integrated, "fallback_reason"] = "integrated late"
paper_fallback_transfers.loc[is_pending, "fallback_reason"] = "never actioned"
paper_fallback_transfers.loc[has_sender_error, "fallback_reason"] = "sender reported failure"
paper_fallback_transfers.loc[has_requestor_error, "fallback_reason"] = "requestor reported failure"
paper_fallback_transfers["fallback_reason"].value_counts()


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


integrated late                12413
never actioned                  5423
requestor reported failure      4719
sender reported failure         3542
other uncatagorised failure       13
Name: fallback_reason, dtype: int64

In [20]:
paper_fallback_transfers[["sending_supplier", "requesting_supplier", "fallback_reason"]]\
    .pivot_table(
        index=[
            "sending_supplier", "requesting_supplier",
        ],
        columns=["fallback_reason"],
        aggfunc=len,
        fill_value=0)

fallback_reason                       integrated late  never actioned  \
sending_supplier requesting_supplier                                    
EMIS             EMIS                            7683            1726   
                 Microtest                          1              19   
                 TPP                             2388            1274   
                 Unknown                            5               3   
                 Vision                            71             111   
Microtest        EMIS                               0              42   
                 Microtest                          0              11   
                 TPP                                0              21   
                 Vision                             0               2   
TPP              EMIS                            2084             446   
                 Microtest                          0               6   
                 TPP                                0               1   
                 Unknown                            1               7   
                 Vision                             2               5   
Unknown          EMIS                              17              19   
                 TPP                                6              15   
Vision           EMIS                             103            1190   
                 Microtest                          0               1   
                 TPP                               21             242   
                 Vision                            31             282   

fallback_reason                       other uncatagorised failure  \
sending_supplier requesting_supplier                                
EMIS             EMIS                                           9   
                 Microtest                                      0   
                 TPP                                            1   
                 Unknown                                        0   
                 Vision                                         0   
Microtest        EMIS                                           0   
                 Microtest                                      0   
                 TPP                                            1   
                 Vision                                         0   
TPP              EMIS                                           2   
                 Microtest                                      0   
                 TPP                                            0   
                 Unknown                                        0   
                 Vision                                         0   
Unknown          EMIS                                           0   
                 TPP                                            0   
Vision           EMIS                                           0   
                 Microtest                                      0   
                 TPP                                            0   
                 Vision                                         0   

fallback_reason                       requestor reported failure  \
sending_supplier requesting_supplier                               
EMIS             EMIS                                       1301   
                 Microtest                                     0   
                 TPP                                        1153   
                 Unknown                                       1   
                 Vision                                       42   
Microtest        EMIS                                          0   
                 Microtest                                     0   
                 TPP                                           0   
                 Vision                                        0   
TPP              EMIS                                       2167   
                 Microtest                                

In [15]:
# When sender and requestor both report an error, what is the sender error
paper_fallback_transfers[has_sender_error & has_requestor_error]["sender_error_name"].value_counts()

Spine error         122
Unauthorised req      5
Name: sender_error_name, dtype: int64

In [17]:
paper_fallback_transfers\
    .loc[has_sender_error, ["sending_supplier", "requesting_supplier", "sender_error_name"]]\
    .pivot_table(
        index=[
            "sending_supplier", "requesting_supplier",
        ],
        columns=["sender_error_name"],
        aggfunc=len,
        fill_value=0)

sender_error_name                     Extract malformed  Failed to generate  \
sending_supplier requesting_supplier                                          
EMIS             EMIS                                 1                 326   
                 Microtest                            0                   0   
                 TPP                                  0                 106   
                 Vision                               0                   4   
Microtest        EMIS                                 0                   8   
                 Microtest                            0                   3   
                 TPP                                  0                  12   
TPP              EMIS                                 0                   0   
                 Microtest                            0                   0   
                 TPP                                  0                   0   
                 Unknown                              0                   0   
                 Vision                               0                   0   
Unknown          EMIS                                 0                   0   
Vision           EMIS                                 0                   0   
                 Vision                               0                   0   

sender_error_name                     GP2GP disabled  LM general failure  \
sending_supplier requesting_supplier                                       
EMIS             EMIS                             12                   0   
                 Microtest                         0                   0   
                 TPP                               5                   0   
                 Vision                            0                   0   
Microtest        EMIS                              0                   0   
                 Microtest                         0                   0   
                 TPP                               0                   0   
TPP              EMIS                             10                1097   
                 Microtest                         0                   3   
                 TPP                               3                   0   
                 Unknown                          37                   4   
                 Vision                            0                  19   
Unknown          EMIS                              1                   6   
Vision           EMIS                              0                   0   
                 Vision                            0                   0   

sender_error_name                     Not at surgery  Req not LM compliant  \
sending_supplier requesting_supplier                                         
EMIS             EMIS                             96                     2   
                 Microtest                         0                    53   
                 TPP                              24                     3   
                 Vision                            1                   868   
Microtest        EMIS                              0                     0   
                 Microtest                         0                     0   
                 TPP                               0                     0   
TPP              EMIS                             34                     2   
                 Microtest                         0                    43   
                 TPP                               1                     0   
                 Unknown                           0                     0   
                 Vision                            0                   372   
Unknown          EMIS                              0                     0   
Vision           EMIS                              5                     0   
                 Vision                            1                     0   

sender_error_name                   

In [18]:
paper_fallback_transfers\
    .loc[has_requestor_error, ["sending_supplier", "requesting_supplier", "requestor_error_name"]]\
    .pivot_table(
        index=[
            "sending_supplier", "requesting_supplier",
        ],
        columns=["requestor_error_name"],
        aggfunc=len,
        fill_value=0
)

requestor_error_name                  ABA supressed  ABA wrong patint  \
sending_supplier requesting_supplier                                    
EMIS             EMIS                           710                44   
                 TPP                              0                 1   
                 Unknown                          0                 0   
                 Vision                           0                 0   
TPP              EMIS                           182                21   
                 Unknown                          0                 0   
                 Vision                           0                 0   
Unknown          EMIS                             2                 0   
                 TPP                              0                 0   
Vision           EMIS                            13                 0   
                 TPP                              0                 0   

requestor_error_name                  Duplicate EHR  Extract malformed  \
sending_supplier requesting_supplier                                     
EMIS             EMIS                           181                  2   
                 TPP                            640                  0   
                 Unknown                          0                  0   
                 Vision                          31                  0   
TPP              EMIS                            19                  1   
                 Unknown                          0                  0   
                 Vision                           1                  0   
Unknown          EMIS                             2                  0   
                 TPP                              5                  0   
Vision           EMIS                             0                 13   
                 TPP                              0                  0   

requestor_error_name                  Failed to integrate  \
sending_supplier requesting_supplier                        
EMIS             EMIS                                  17   
                 TPP                                   62   
                 Unknown                                0   
                 Vision                                11   
TPP              EMIS                                   0   
                 Unknown                                0   
                 Vision                                 3   
Unknown          EMIS                                   0   
                 TPP                                    0   
Vision           EMIS                                   1   
                 TPP                                    1   

requestor_error_name                  Filed as attachment  LM general failure  \
sending_supplier requesting_supplier                                            
EMIS             EMIS                                   9                   0   
                 TPP                                    0                 290   
                 Unknown                                0                   1   
                 Vision                                 0                   0   
TPP              EMIS                                   2                   0   
                 Unknown                                0                   0   
                 Vision                                 0                   0   
Unknown          EMIS                                   0                   0   
                 TPP                                    0                   1   
Vision           EMIS                                   0                   0   
                 TPP                                    0                   0   

requestor_error_name                  Missing LM  Spine error  Timeout  \
sending_supplier requesting_supplier                                     
EMIS             EMIS                         76            0       58   
                 